### Convert time stamp strings in tweet_attribute and user_attribute dataframes to pandas datetime format.
### Reverse geocode tweets and save results in two new columns 'State' and 'County'

In [1]:
import json
import pandas as pd
import numpy as np
import csv
import re
from time import sleep
import sys

In [9]:
def timestamp_to_datetime(df):
    '''Converts time stamp strings to pandas datetime. Process data in chunks of 5000 rows'''
    datetimes = []
    count = 0
    for k,g in df.groupby(np.arange(len(df))//5000):
        datetimes.append(pd.to_datetime(g, infer_datetime_format=True))
        sys.stdout.write('\r')
        sys.stdout.write("[%-50s] %d%%" % ('='*np.int(k*50.0/(len(df)//5000)), np.int(k*100.0/(len(df)//5000))))
        sys.stdout.flush()
                
        #if (k > 20):
        #    break
        
    sys.stdout.write('\r')
    sys.stdout.write("[%-50s] %d%%" % ('='*50, 100))
    
    return datetimes

In [2]:
# Load tweet attribute data
tweet_attributes_file = 'sandy_tweets_attributes.csv'
df = pd.read_csv(tweet_attributes_file)
print (len(df))
df.head()

4779087


,tweet_id,user_id,retweet_count,time_stamp,longitude,latitude
0,260244125050363904,295902181,0,Mon Oct 22 05:00:09 +0000 2012,-74.078101,40.735218
1,260244177412042752,85314436,0,Mon Oct 22 05:00:21 +0000 2012,-81.505790,33.460462
2,260244177105850368,239968255,0,Mon Oct 22 05:00:21 +0000 2012,-77.099999,39.344184
3,260244156729942016,703352862,0,Mon Oct 22 05:00:17 +0000 2012,-80.907470,39.618102
4,260244145694728192,581488152,0,Mon Oct 22 05:00:14 +0000 2012,-76.579826,39.816450


In [10]:
datetimes = timestamp_to_datetime(df['time_stamp'])
datetimes_1 = np.concatenate(datetimes).ravel()
df['time_stamp'] = datetimes_1
df.head()

[==================================================] 100%

,tweet_id,user_id,retweet_count,time_stamp,longitude,latitude
0,260244125050363904,295902181,0,2012-10-22 05:00:09,-74.078101,40.735218
1,260244177412042752,85314436,0,2012-10-22 05:00:21,-81.505790,33.460462
2,260244177105850368,239968255,0,2012-10-22 05:00:21,-77.099999,39.344184
3,260244156729942016,703352862,0,2012-10-22 05:00:17,-80.907470,39.618102
4,260244145694728192,581488152,0,2012-10-22 05:00:14,-76.579826,39.816450


In [11]:
# Keep only geo located tweets
print ('%d lines contain NaN: ' % (len(df) - len(df.dropna())))
df = df.dropna()

0 lines contain NaN: 


In [12]:
# In the following, we use 'reverse-geocoder' package downloaded from Ref.: https://github.com/thampiman/reverse-geocoder

import io
import reverse_geocoder as rg

coordinates = df[['latitude', 'longitude']]
tuples = tuple([tuple(x) for x in coordinates.values])
         # Convert DataFrame columns to tuple of tuples
rgdata = rg.search(tuples)

rows_state, rows_county = [], []
for idx, rg in enumerate(rgdata):
    rows_state.append(rg['admin1'])
    rows_county.append(rg['admin2'])    
    
df['state'], df['county'] = rows_state, rows_county
df.head()

Loading formatted geocoded file...


,tweet_id,user_id,retweet_count,time_stamp,longitude,latitude,state,county
0,260244125050363904,295902181,0,2012-10-22 05:00:09,-74.078101,40.735218,New Jersey,Hudson County
1,260244177412042752,85314436,0,2012-10-22 05:00:21,-81.505790,33.460462,South Carolina,Barnwell County
2,260244177105850368,239968255,0,2012-10-22 05:00:21,-77.099999,39.344184,Maryland,Carroll County
3,260244156729942016,703352862,0,2012-10-22 05:00:17,-80.907470,39.618102,West Virginia,Tyler County
4,260244145694728192,581488152,0,2012-10-22 05:00:14,-76.579826,39.816450,Pennsylvania,York County


In [13]:
header = ['tweet_id', 'user_id', 'retweet_count', 'time_stamp', 'longitude', 'latitude', 'state', 'county']
df.to_csv('sandy_tweets_attributes_rev_geocoded_formatted_timestamps.csv', columns = header)

### Tests

In [14]:
print (np.mean(df[df['state'] == 'New York']['longitude']))
print (np.std(df[df['state'] == 'New York']['longitude']))

print (np.mean(df[df['state'] == 'New York']['latitude']))
print (np.std(df[df['state'] == 'New York']['latitude']))

print('')
print (np.mean(df[df['state'] == 'Washington, D.C.']['longitude']))
print (np.std(df[df['state'] == 'Washington, D.C.']['longitude']))

print (np.mean(df[df['state'] == 'Washington, D.C.']['latitude']))
print (np.std(df[df['state'] == 'Washington, D.C.']['latitude']))

print('')
print (np.mean(df[df['state'] == 'Ontario']['longitude']))
print (np.std(df[df['state'] == 'Ontario']['longitude']))

print (np.mean(df[df['state'] == 'Ontario']['latitude']))
print (np.std(df[df['state'] == 'Ontario']['latitude']))

-74.8904834714
1.83665553201
41.5780526566
1.0381112744

-77.0290102174
0.018302702135
38.9021835838
0.0178535390411

-75.9904787133
1.62343961431
44.3878491984
0.800220774933


In [15]:
#df[df['state'] == 'Ontario']
#df[df['state'] == 'Washington, D.C.']
df[df['state'] == 'New York']

,tweet_id,user_id,retweet_count,time_stamp,longitude,latitude,state,county
10,260244141156491264,98974320,0,2012-10-22 05:00:13,-79.339525,42.449143,New York,Chautauqua County
17,260244114883366912,456105010,0,2012-10-22 05:00:06,-73.738550,40.726990,New York,Nassau County
35,260244142540607488,46308177,0,2012-10-22 05:00:13,-74.106378,40.609279,New York,Richmond County
42,260244156029497345,107547858,0,2012-10-22 05:00:16,-74.146818,40.564605,New York,Richmond County
46,260244098504589312,790818829,0,2012-10-22 05:00:02,-73.842642,40.916031,New York,Westchester County
78,260244242352459776,350287015,1,2012-10-22 05:00:37,-76.078670,42.088634,New York,Broome County
84,260244187461603328,360395069,2,2012-10-22 05:00:24,-76.285808,42.499914,New York,Tompkins County
88,260244213164277760,621111983,0,2012-10-22 05:00:30,-76.995330,43.066734,New York,Wayne County
94,260244187956514817,338830308,0,2012-10-22 05:00:24,-75.266677,43.082288,New York,Oneida County
97,260244200010948608,198010784,0,2012-10-22 05:00:27,-73.906342,40.846667,New York,New York County


### User timestamps

In [16]:
# Load user attribute data
user_attributes_file = 'sandy_tweets_user_attributes.csv'
df_user = pd.read_csv(user_attributes_file)
print (len(df_user))
df_user.head()

4779087


,user_id,followers_count,friends_count,statuses_count,screen_name,name,created_at
0,295902181,263,71,22153,_CornelliaM,HILARY_BANKZ,Mon May 09 21:17:32 +0000 2011
1,85314436,3377,3462,28147,synclaire_xoxo,Tara Synclaire,Mon Oct 26 13:26:43 +0000 2009
2,239968255,753,762,16182,OnlyAprilMoreno,ace ♠️,Tue Jan 18 21:21:42 +0000 2011
3,703352862,114,146,154,KameronMcclain,Kammm (:,Wed Jul 18 16:19:19 +0000 2012
4,581488152,393,550,13538,lady_lauer15,Natasha Lauer,Wed May 16 01:21:49 +0000 2012


In [17]:
datetimes = timestamp_to_datetime(df_user['created_at'])
datetimes_1 = np.concatenate(datetimes).ravel()
df_user['created_at'] = datetimes_1
df_user.head()

[==================================================] 100%

,user_id,followers_count,friends_count,statuses_count,screen_name,name,created_at
0,295902181,263,71,22153,_CornelliaM,HILARY_BANKZ,2011-05-09 21:17:32
1,85314436,3377,3462,28147,synclaire_xoxo,Tara Synclaire,2009-10-26 13:26:43
2,239968255,753,762,16182,OnlyAprilMoreno,ace ♠️,2011-01-18 21:21:42
3,703352862,114,146,154,KameronMcclain,Kammm (:,2012-07-18 16:19:19
4,581488152,393,550,13538,lady_lauer15,Natasha Lauer,2012-05-16 01:21:49


In [22]:
header = ['user_id', 'followers_count', 'friends_count', 'statuses_count',
          'screen_name', 'name', 'created_at']
df_user.to_csv('sandy_tweets_user_attributes_formatted_timestamps.csv', columns=header)

In [21]:
df_user.columns

Index([u'user_id', u'followers_count', u'friends_count', u'statuses_count',
       u'screen_name', u'name', u'created_at'],
      dtype='object')